In [11]:
from bs4 import BeautifulSoup
from collections import OrderedDict

In [2]:
# TODO: set from cli
with open("demo-files/sample-0-origin.html") as f:
    html = BeautifulSoup(f, 'html.parser')


In [3]:
# TODO: set from cli
id_ = 'make-everything-ok-button'

In [4]:
elements = html.select('#'+id_)
# TODO: if element not found - raise exc

initial_element = elements[0]
initial_element

<a class="btn btn-success" href="#ok" id="make-everything-ok-button" onclick="javascript:window.okDone(); return false;" rel="next" title="Make-Button">
                              Make everything OK
                            </a>

In [5]:
attributes_to_match = {}
for attr, value in initial_element.attrs.items():
    if type(value) == list:
        value = ' '.join(value)
    attributes_to_match[attr] = value
    
text_to_match = initial_element.text.strip()
attributes_to_match

{'id': 'make-everything-ok-button',
 'class': 'btn btn-success',
 'href': '#ok',
 'title': 'Make-Button',
 'rel': 'next',
 'onclick': 'javascript:window.okDone(); return false;'}

In [7]:
# ------------
# now open target file and search for elements having attributes_to_match
# with open("demo-files/sample-1-evil-gemini.html") as f:
with open("demo-files/sample-2-container-and-clone.html") as f:
    target_html = BeautifulSoup(f, 'html.parser')

def find_tag_by_text(html, text):
    # In BeautifulSoup, 'text' propery holds text of all child elements,
    #  so all parents of actual element will be returned (beginning with <html> tag)
    # That's why we take last element
    tags = html.find_all(lambda tag: text in tag.text)
    # Return list just for simplicity of usage in current program.
    # In real world, its better to return None or just tag
    return [] if len(tags) == 0 else [tags[-1]] 

matches = {}
for attr, value in attributes_to_match.items():
    selector = f'{attr}="{value}"'
    matches[selector] = target_html.select(f'[{selector}]')
matches[f'text="{text_to_match}"'] = find_tag_by_text(target_html, text_to_match)

matches

{'id="make-everything-ok-button"': [],
 'class="btn btn-success"': [],
 'href="#ok"': [<a class="btn test-link-ok" href="#ok" onclick="javascript:window.okComplete(); return false;" rel="next" title="Make-Button">
                                Make everything OK
                              </a>],
 'title="Make-Button"': [<a class="btn test-link-ok" href="#ok" onclick="javascript:window.okComplete(); return false;" rel="next" title="Make-Button">
                                Make everything OK
                              </a>,
  <a class="btn btn-info" href="#incorrect-link" onclick="javascript:window.close(); return false;" style="display:none" title="Make-Button">
                                Make everything OK
                              </a>],
 'rel="next"': [<a class="btn test-link-ok" href="#ok" onclick="javascript:window.okComplete(); return false;" rel="next" title="Make-Button">
                                Make everything OK
                              </a>]

In [9]:
temp_all_matched_tags = [] 
for v in matches.values(): 
    temp_all_matched_tags.extend(v)
all_matched_tags = set(temp_all_matched_tags)
all_matched_tags


{<a class="btn btn-info" href="#incorrect-link" onclick="javascript:window.close(); return false;" style="display:none" title="Make-Button">
                               Make everything OK
                             </a>,
 <a class="btn test-link-ok" href="#ok" onclick="javascript:window.okComplete(); return false;" rel="next" title="Make-Button">
                               Make everything OK
                             </a>}

In [10]:
matches_per_tag = {}
for tag in all_matched_tags:
    matched_selectors = [selector for selector in matches.keys() if tag in matches[selector]]
    matches_per_tag[tag] = matched_selectors
matches_per_tag

{<a class="btn test-link-ok" href="#ok" onclick="javascript:window.okComplete(); return false;" rel="next" title="Make-Button">
                               Make everything OK
                             </a>: ['href="#ok"',
  'title="Make-Button"',
  'rel="next"'],
 <a class="btn btn-info" href="#incorrect-link" onclick="javascript:window.close(); return false;" style="display:none" title="Make-Button">
                               Make everything OK
                             </a>: ['title="Make-Button"',
  'text="Make everything OK"']}

In [20]:
# find tag with maximum number of matched selectors
best_tag = max(matches_per_tag, key = lambda tag: len(matches_per_tag[tag]))
print(best_tag)
matches_per_tag[best_tag]

<a class="btn test-link-ok" href="#ok" onclick="javascript:window.okComplete(); return false;" rel="next" title="Make-Button">
                              Make everything OK
                            </a>


['href="#ok"', 'title="Make-Button"', 'rel="next"']